In [1]:
import pandas as pd
from itertools import chain

development = pd.read_csv("./../../DSL_Winter_Project_2024/development.csv")

outlier_column_index=[0, 7, 12, 15, 16, 17]
columns_to_drop=[]

for index in outlier_column_index:
    columns_to_drop.append('pmax[%s]' % index)
    columns_to_drop.append('negpmax[%s]' % index)
    columns_to_drop.append('tmax[%s]' % index)
    columns_to_drop.append('area[%s]' % index)
    columns_to_drop.append('rms[%s]' % index)

df=development.drop(columns=columns_to_drop)

train_datasets = []
test_datasets=[]

for i in range(0, len(df), 100):
    subset1 = df.iloc[i:i+100].head(3)
    train_datasets.append(subset1)

    subset2 = df.iloc[i:i+100].tail(1)
    test_datasets.append(subset2)

train_df=pd.concat(train_datasets, ignore_index=False)
test_df=pd.concat(test_datasets, ignore_index=False)

In [2]:
X_train=train_df.drop(columns=['x', 'y'])
y_train=train_df.loc[:,['x', 'y']]

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(random_state=0)
)

regr_multirf.fit(X_train, y_train)
regr_rf = RandomForestRegressor(random_state=0)
regr_rf.fit(X_train, y_train)


RandomForestRegressor(random_state=0)

In [4]:
X_test=test_df.drop(columns=['x', 'y'])
y_multirf = regr_multirf.predict(X_test)
y_rf = regr_rf.predict(X_test)

In [5]:
y_test=test_df.loc[:,['x', 'y']]


In [15]:
regr_rf.score(X_test, y_test)

0.9972609583910332

# compare distance

In [6]:
import numpy as np
distance1=np.mean(np.sqrt(np.sum((y_multirf - y_test) ** 2, axis=1)))
distance1

7.293157121441379

In [7]:
distance2=np.mean(np.sqrt(np.sum((y_rf - y_test) ** 2, axis=1)))
distance2

6.214035275865451

In [8]:
from sklearn.model_selection import cross_val_score
score_pre1 = cross_val_score(regr_multirf, X_train, y_train, cv = 10).mean()
score_pre1

-0.5014419998664159

In [9]:
from sklearn.model_selection import cross_val_score

score_pre2 = cross_val_score(regr_rf, X_train, y_train, cv = 10).mean()
score_pre2

-0.9082794664414744

In [10]:
import numpy as np
n_estimators_range=[int(x) for x in np.linspace(start=50,stop=3000,num=60)]
max_features_range=['auto','sqrt','log2']
max_depth_range=[int(x) for x in np.linspace(10,500,num=50)]
max_depth_range.append(None)
min_samples_split_range=[2,5,10]
min_samples_leaf_range=[1,2,4,8]

rfr_hp_range={'n_estimators':n_estimators_range,
              'max_features':max_features_range,
              'max_depth':max_depth_range,
              'min_samples_split':min_samples_split_range,
              'min_samples_leaf':min_samples_leaf_range
             }
print(rfr_hp_range)

{'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 8]}


In [11]:
from sklearn.model_selection import RandomizedSearchCV

rfr_base=RandomForestRegressor()
rfr_random=RandomizedSearchCV(estimator=rfr_base,
                              param_distributions=rfr_hp_range,
                              n_iter=200,
                              n_jobs=-1,
                              cv=3,
                              verbose=1,
                              random_state=42)
rfr_random.fit(X_train,y_train)

best_hp_now=rfr_random.best_params_
print(best_hp_now)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


/Users/xuyan/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/xuyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
231 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
128 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/xuyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/xuyan/anaconda

{'n_estimators': 350, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 270}


In [12]:
best_hp_now

{'n_estimators': 350,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 270}

In [13]:
best = RandomForestRegressor(n_estimators=350,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=270)
best.fit(X_train,y_train)
best.score(X_test,y_test)

0.9973878965622149

In [14]:
best_y = best.predict(X_test)

distance=np.mean(np.sqrt(np.sum((best_y - y_test) ** 2, axis=1)))
distance

6.425934334910736

In [16]:
from sklearn.model_selection import GridSearchCV
rfr_hp_range_2={'n_estimators':[1600,1650,1700],
                'max_depth':[320,330,340],
                }
rfr_2_base=RandomForestRegressor()
rfr_2_random=GridSearchCV(estimator=rfr_2_base,
                          param_grid=rfr_hp_range_2,
                          cv=3,
                          verbose=1,
                          n_jobs=-1)
rfr_2_random.fit(X_train,y_train)

best_hp_now_2=rfr_2_random.best_params_

print(best_hp_now_2)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
{'max_depth': 320, 'n_estimators': 1700}


In [17]:
best = RandomForestRegressor(n_estimators=1700,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=320)
best.fit(X_train,y_train)
best.score(X_test,y_test)

0.9974066491851251

In [18]:
best_y = best.predict(X_test)

distance=np.mean(np.sqrt(np.sum((best_y - y_test) ** 2, axis=1)))
distance

6.374090190804557

In [19]:
rfr_hp_range_3={'n_estimators':n_estimators_range,
                'max_depth':max_depth_range,
                }
rfr_3_base=RandomForestRegressor()
rfr_3_random=GridSearchCV(estimator=rfr_3_base,
                          param_grid=rfr_hp_range_3,
                          cv=3,
                          verbose=1,
                          n_jobs=-1)
rfr_3_random.fit(X_train,y_train)

best_hp_now_3=rfr_3_random.best_params_

print(best_hp_now_3)

Fitting 3 folds for each of 3060 candidates, totalling 9180 fits


/Users/xuyan/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
